### Excel data

In [8]:
import pandas as pd

df = pd.read_excel("data/raw/Координаты.xlsx")[:13]
df.columns = ['Положение', 'Координаты центра X, Y', 'Угол поворота в градусах (Относительно центра образца)', 'Размер заготовки, мм']

In [9]:
df.index = df['Положение'].str.slice(10).astype(int)
df.index.name = 'position'

In [10]:
df['Координаты центра X, Y'] = df['Координаты центра X, Y'].apply(eval)
df['coord_x'] = df['Координаты центра X, Y'].apply(lambda coords: coords[0])
df['coord_y'] = df['Координаты центра X, Y'].apply(lambda coords: coords[1])

In [11]:
df.rename(columns={'Угол поворота в градусах (Относительно центра образца)': 'degree', 'Размер заготовки, мм': 'size'}, inplace=True)
df.drop(['Положение', 'Координаты центра X, Y'], axis=1, inplace=True)

In [12]:
df.to_csv("data/Координаты.csv")

### Images

In [1]:
import os, shutil
import pandas as pd

In [2]:
data = []
for i in range(14):
    folder = f'data/raw/Положение {i}'

    for image_name in os.listdir(folder):
        data.append((folder[folder.rfind('/') + 1:], image_name[:-4]))

df = pd.DataFrame(data, columns=['position', 'file_name'])

In [3]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(df, test_size=0.2, stratify=df['position'])

In [4]:
os.makedirs('yolo/data/images/train', exist_ok=True)
os.makedirs('yolo/data/labels/train', exist_ok=True)

for i in range(train.shape[0]):
    data = train.iloc[i]
    shutil.copy(
        f'data/raw/{data["position"]}/{data["file_name"]}.bmp',
        f'yolo/data/images/train/{i}.bmp'
    )

    shutil.copy(
        f'data/raw/Annotations/{data["position"]}/{data["file_name"]}.txt',
        f'yolo/data/labels/train/{i}.txt'
    )

os.makedirs('yolo/data/images/val', exist_ok=True)
os.makedirs('yolo/data/labels/val', exist_ok=True)

for i in range(valid.shape[0]):
    data = valid.iloc[i]
    shutil.copy(
        f'data/raw/{data["position"]}/{data["file_name"]}.bmp',
        f'yolo/data/images/val/{i}.bmp'
    )

    shutil.copy(
        f'data/raw/Annotations/{data["position"]}/{data["file_name"]}.txt',
        f'yolo/data/labels/val/{i}.txt'
    )